<a href="https://colab.research.google.com/github/sylwiakantor/AirlyAPI/blob/main/Kopia_notatnika_MPSiS_C04_programowanie_dyskretne_wst%C4%99p_PuLP__ONLINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPSiS - ćwiczenia 4: *Wstęp do programowania dyskretnego*. Materiał przygotowawczy do ćwiczeń

**CEL**: zapoznanie z implementowaniem zadań programowania matematycznego, w których używamy nie tylko zmiennych ciągłych, ale też zmiennych dyskretnych.

Materiał jest dostępny od 31.10.2022. Link do notebooka z rozwiązaniami należy odesłać w odpowiedniej [aktywności UPEL](https://upel.agh.edu.pl/mod/assign/view.php?id=60446) do 07.11.2022, godz. 8.00. Proszę **nie usuwać** uzyskanych danych wyjściowych!

Do wykonania jest **jedno** zadanie. Na potrzeby rozwiązania oczywiście można używać dowolnych dodatkowych pakietów Pythona. Ale oczywiście implementacja zadań powinna być dokonywana z użyciem `PuLP`, a nie np. `docplex` itp.

Wszelkie wątpliwości dotyczące notebooka, w tym sposobu rozwiązania zadań, można dyskutować publicznie - w szczególności użycie [forum](https://upel.agh.edu.pl/mod/forum/view.php?id=1512) zwiększa szansę, że prowadzący pomoże!

## Przygotowanie środowiska

W celu wykonania zadań w notebooku należy go sobie skopiować do lokalnego konta Google. Następnie należy udostępnić prowadzącemu **link** do własnego notebooka (proszę ustawić prawa edycji dla osoby posługującej się tym linkiem!). Proszę nie wysyłać plików *.ipynb czy tp.

W przypadku wykonywania zadań w zespole proszę umieścić odpowiednią informację (nt. składu zespołu) w pierwszej komórce tekstowej notebooka.

## Wstęp

W praktyce bez użycia programowania dyskretnego (czyli modelowania optymalizacyjnego, które angażuje zmienne całkowitoliczbowe, binarne lub po prostu używające dyskretnego - typowo: skończonego - zbioru wartości) nie da się modelować realistycznych zagadnień projektowania sieci i systemów. Z tego względu ćwiczeniu modelowania z użyciem zmiennych dyskretnych będą poświęcone trzy zajęcia.

Należy zapoznać się z:

*  [Problemem plecakowym](https://en.wikipedia.org/wiki/Knapsack_problem) (ang. *knapsack problem*). Mimo niespecjalnie sieciowego brzmienia jest to paradygmatyczny problem, którego rozszerzenia są stosowane do opisywania alokacji zasobów w systemach informatycznych. Części z Państwa problem może być znany z przedmiotu *Algorytmy i struktury danych* (ze względu na fakt, że problem plecakowy to paradygmatyczny problem dyskretny; poza tym on ma znane rozwiązanie pseudowielomianowe oparte na tzw. programowaniu dynamicznym oraz istnieje dla niego wiele heurystyk, algorytmów przybliżonycy/aproksymacyjnych itd.).
*  Treścią konspektu nr 4: *Programowanie dyskretne w projektowaniu sieci i systemów* (on może nie być w pełni jasny przed Wykładem 6; warto jednak **tylko rzucić okiem**, żeby zobaczyć, do czego służy modelowanie dyskretne w sieciach).

W tygodniu, w którym wykonują Państwo to ćwiczenie (tj. zadanie domowe przed zajęciami), odbędzie się Wykład 6, w ramach którego zostanie zrealizowany początkowy materiał z konspektu nr 4. Z tego powodu niniejsze ćwiczenia oraz zadania do rozwiązania są niedługie i mają tylko ilustracyjny charakter. W dużym stopniu wymagają głównie zrozumienia przedstawionego kodu oraz zastanowienia się nad rozwiązaniem.

### Test-wejściówka na rozpoczęcie zajęć

W trakcie pierwszych **5** min. ćwiczeń odbędzie się test sprawdzający opanowanie wybranego materiału z przedmiotu.
*   Test odbędzie się za pośrednictwem platformy UPEL.
*   W przypadku spóźnienia się **nie ma możliwości**  podejścia do testu w innym czasie, terminie, przedłużenia trwania itp.
*   Test ma formę pięciu pytań jednokrotnego wyboru.
*   Test jest tak skonstruowany, że po przejściu do kolejnego pytania nie da się powrócić do wcześniejszego pytania.
*   Punkty z tego testu maję charakter bonusowy.

Test obejmuje materiał z całości Wykładu 5 *Problemy alokacji zasobów i wymiarowania sieci* (konspekt nr 3) oraz Wykładu 6 *Programowanie dyskretne w projektowaniu sieci i systemów* (tylko w części odpowiadającej tym elementom konspektu nr 4, które zostaną zrealizowane na wykładzie w dn. 03.11.2022).

### Do przygotowania na zajęcia na żywo

Poza przygotowaniem się z Wykładu 6 oraz wykonaniem ćwiczeń/zadania z niniejszego notebooka, żadne inne przygotowanie nie jest potrzebne. Przyda się znajomość następujących sformułowań:

*   		Różne wersje [problemu plecakowego](https://en.wikipedia.org/wiki/Knapsack_problem) (ang. *knapsack problem*), tj. jednowymiarowy, wielowymiarowy, uogólniony.
*       Problem projektowania topologii z rozkładem przepływów.
*   		[Problem przydziału](https://en.wikipedia.org/wiki/Assignment_problem) (ang. *assignment problem*).
*       [Problem komiwojażera](https://en.wikipedia.org/wiki/Travelling_salesman_problem) (ang. *traveling salesman problem*).

Warto jak zwykle zadbać o środowisko pracy, jeśli chce się pracować na własnym komputerze.

In [1]:
#@title Biblioteki
#@markdown * Tutaj użyjemy sobie specyficznego obiektu `namedtuple` do obsługi danych
#@markdown * Mogą się nam też przydać różne solwery

# Najpierw instalujemy solwery, potem importujemy PuLP!
! apt-get install coinor-cbc  # CoinOR CBC
! pip install cplex           # CPLEX
! pip install gurobipy        # Gurobi

from collections import namedtuple

import pprint

! pip install pulp
import pulp as plp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 4 not upgraded.
Need to get 2,737 kB of archives.
After this operation, 8,130 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libcoinutils3v5 amd64 2.10.14+repack1-1 [472 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libosi1v5 amd64 0.107.9+repack1-1 [270 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libclp1 amd64 1.16

## Ćwiczenia

## Problem plecakowy

Zaczniemy od implementacji prostego modelu z użyciem zmiennych dyskretnych. Użyjemy jako przykładowego problemu plecakowego.

### Problem plecakowy - werbalizacja zagadnienia:

1. Nasze zadanie polega na jak najlepszym użyciu plecaka (powiedzmy, że jesteśmy przemytnikiem i chcemy wiedzieć, co nam się najbardziej opłaci przy przenoszeniu różnych rzeczy przez zieloną granicę - tam sprzedamy przeniesione rzeczy na czarnym rynku).
2. Plecak ma określoną nośność, tj. nie możemy do niego zapakować zbyt dużo rzeczy.
3. Każda rzecz, którą możemy wrzucić do plecaka, ma swój ciężar (wagę, masę).
4. Dysponujemy też określoną liczbą rzeczy różnego rodzaju (tj. nie możemy ich zapakować, ile nam się podoba).
5. Chcemy użyć plecaka tak, żeby zabrać jak najcenniesze rzeczy. W przypadku każdej z rzeczy znamy jej wartość (użyteczność).

In [ ]:
#@markdown Dane

# Jeśli nośność jest wyrażona w kg, to zakładamy że potem ciężar różnych rzeczy też wyrażony w kg
NOSNOSC_PLECAKA = 4

RZECZY = [ # "Nazwa", "Masa", "Liczba_max", "Wartosc"
    ("Czekolada", 0.1, 10, 10),
    ("Piwerko", 0.5, 5, 3),
    ("Советское игристое", 2, 1, 30),
    ("Kiełbacha", 0.35, 3, 7),
    ("Jabłka", 1, 10, 1),
    ("Papierochy", 0.1, 100, 0.5)
]

Jak to rozwiązać?

### Problem plecakowy - zadanie optymalizacji:

W celu zaimplementowania modelu musimy oczywiście najpierw przygotować odpowiedni program matematyczny:

*   Indeksy: $ i = 1, 2, \dots, N $: indeksujemy rzeczy, $N$ to liczba dostępnych rzeczy (w przykładowych danych $N=6$)
*   Stałe:
  *   $ W $: nośność plecaka (w przykładowych danych $W=4$; powiedzmy że domyślnie chodzi o 4 kg)
  *   $ M(i) $: waga/ciężar rzeczy $i$ (w przykładowych danych $M(\text{Czekolada}) = 0.1$, czyli czekolada waży 100 g)
  *   $ K(i) $: liczba dostępnych rzeczy $i$ (w przykładowych danych $K(\text{Czekolada}) = 10$, tj. możemy włożyć do plecaka co najwyżej 10 czekolad, bo widocznie więcej nie mamy)
  *   $ U(i) $: wartość pojedynczej rzeczy $i$ (w przykładowych danych $U(\text{Czekolada}) = 10$, tj. jedna czekolada widocznie byłaby do sprzedania na rynku w Görlitz za 10 EUR)
*   Zmienne: $ x_i \in \mathbb{Z}_{+} $: mówi nam, jak wiele sztuk rzeczy $i$ włożymy do plecaka (np. $ x_{\text{Czekolada}} = 2$ oznacza że bierzemy dwie czekolady, $ x_{\text{Papierochy}} = 0$ oznacza, że w ogóle nie wkładamy papierosów itd.)
*   Funkcja celu:
$$ \max \, \sum\limits_i U(i)x_i $$
*   Ograniczenia:
  *   Nie możemy przekroczyć nośności plecaka:
  $$ \sum\limits_i M(i)x_i \leq W $$
  *   Nie możemy wziąć większej liczby sztuk rzeczy danego rodzaju niż mamy dostępnych:
  $$ \forall_{i} \quad x_i \leq K(i) $$ 

To sformułowanie wygląda może, jak dostosowane tylko do anegdotki o przemytniku - czy któryś tak w ogóle liczy?) Warto jednak zastanowić się, jak można byłoby je przełożyć na kontekst (tele-)informatyczny (np. zadanie obliczeniowe z określonym kosztem w chmurze).

Klasyczny problem plecakowy to taki, w którym $K(i) = 1$. W przypadku podanych wartości dopuszczamy też inne sytuacje (tj. niekiedy $K(i) > 1$), ale sprawdzimy jak będzie wyglądać rozwiązanie optymalne, jeśli rozwiązemy zadanie w wersji klasycznej i nieklasycznej.

Problem plecakowy z definicji zakłada użycie zmiennych dyskretnych (tj. nie możemy dzielić butelki piwa na pół czy tp., dlatego założyliśmy $x_i \in \mathbb{Z}_{+}$). Sprawdzimy jednak także co się stanie, jeśli dopuścimy uciąglenie zmiennych, tj. tzw. relaksację liniową (gdy pozwolimy $x_i \in \mathbb{R}_{+}$).

Proszę wykonać i przeanalizować podany niżej kod oraz opisy.

In [ ]:
#@markdown Model problemu plecakowego

def knapsack(OBJECTS, MAX_WEIGHT, **kwargs):
  """
  Model problemu plecakowego
    OBJECTS: lista rzeczy potencjalnie wkładanych do plecaka
    MAX_WEIGHT: nośność plecaka
  Można z pomocą dodatkowych flag zmodyfikować uniwersalne sformułowanie problemu plecakowego:
    domyślnie ints = True (używamy zmiennych dyskretnych);
      jeśli wprost zadamy ints = False pozwolimy na uciąglenie wartości zmiennych x
    domyślnie classical = True (pozwalamy jak w klasycznym problemie plecakowym na włożenie co najwyżej jednej rzeczy danego rodzaju);
      jeśli wprost zadamy classical = False wtedy zgadzamy się, że można włożyć więcej rzeczy jednego rodzaju
  """

  dane = namedtuple("Rzecz", ["Nazwa", "Masa", "Liczba_max", "Wartosc"])
  rzeczy = [dane(*o) for o in OBJECTS]
  print("Dane")
  pprint.pprint(rzeczy)
  print()

  model_knapsack = plp.LpProblem(name = 'Problem plecakowy', sense = plp.LpMaximize)

  # Typ użytych zmiennych
  zmienne_calkowitoliczbowe = kwargs.pop('ints', True)
  typ_zmiennej = plp.LpInteger if zmienne_calkowitoliczbowe else plp.LpContinuous

  # Informacja czy mamy problem klasyczny (czyli czy możemy wziąć maksymalnie jedną rzecz danego typu)
  problem_klasyczny = kwargs.pop('classical', True)

  # Definicja zmiennych
  ile_rzeczy = plp.LpVariable.dicts(
      "x",
      [item.Nazwa for item in rzeczy], # indeksowanie po rzeczach (tzn. po ich nazwach)
      lowBound = 0, # niby niepotrzebne, bo mamy maksymalizację, ale warto dodać dla jasności
      cat = typ_zmiennej # typ zmiennej to całkowitoliczbowa (domyślnie) albo uciąglona
  )

  # Funkcja celu
  sumaryczna_wartosc_w_plecaku = plp.lpSum(
      item.Wartosc * ile_rzeczy[item.Nazwa]
        for item in rzeczy
  ) 
  model_knapsack += sumaryczna_wartosc_w_plecaku, "wartość"

  # Ograniczenie na nośność
  sumaryczny_ciezar_plecaka = plp.lpSum(
      item.Masa * ile_rzeczy[item.Nazwa]
        for item in rzeczy
  )
  model_knapsack += sumaryczny_ciezar_plecaka <= MAX_WEIGHT, "nośność"

  # Ograniczenia na liczbę rzeczy
  for item in rzeczy:
    ogr_gorne = 1 if problem_klasyczny else item.Liczba_max
    model_knapsack += ile_rzeczy[item.Nazwa] <= ogr_gorne, f"ile_max_{item.Nazwa}"

  print('-'*5)
  print('Model:')
  print('-'*5)
  print(model_knapsack)

  model_knapsack.solve()

  print('-'*5)
  print('Rozwiązanie')
  print('-'*5)

  if model_knapsack.status:
      print('Znaleziono rozwiązanie optymalne.')
      print('Do plecaka wkładamy:')
      for item in rzeczy:
        if ile_rzeczy[item.Nazwa].varValue:
          print(item.Nazwa, "sztuk: ", ile_rzeczy[item.Nazwa].varValue)
      print("Waga plecaka: ", plp.value(sumaryczny_ciezar_plecaka))
      print("Wartość (użyteczność) plecaka: ", plp.value(sumaryczna_wartosc_w_plecaku))
      print("Czas rozwiązania:", model_knapsack.solutionTime)

  return model_knapsack

W zależności od różnych wartości flag `ints` oraz `classical` w powyższym kodzie na różne sposoby zdefiniowano typ zmiennych, jak również ograniczenie górne na wartości, które mogą przyjmować. Poniżej rozpisano odpowiednie fragmenty kodu w taki sposób, w jaki można byłoby go zadać, gdybyśmy z góry mieli ustalone konkretne kombinacje flag.

Mamy cztery możliwości:
*   Wersja pełna problemu plecakowego (nieklasyczna, bo zmienne nie są binarne, ale w ogóle całkowitoliczbowe), zgodna z podanym wyżej sformułowaniem matematycznym (`ints = True` oraz `classical = False`): 
```python
  ile_rzeczy = plp.LpVariable.dicts(
      "x",
      [item.Nazwa for item in rzeczy],
      lowBound = 0,
      cat = plp.LpInteger
  )

  ...

  for item in rzeczy:
      model_knapsack += ile_rzeczy[item.Nazwa] <= item.Liczba_max, f"ile_max_{item.Nazwa}"
```
*   Wersja klasyczna problemu plecakowego - czyli wersja binarna (`ints = True` oraz `classical = True`):
```python
  ile_rzeczy = plp.LpVariable(
      "x",
      [item.Nazwa for item in rzeczy],
      cat = plp.LpBinary
  )
```
Tutaj ze względu na binarność zmiennych ograniczenia:
```python
    for item in rzeczy:
      model_knapsack += ile_rzeczy[item.Nazwa] <= 1, f"ile_max_{item.Nazwa}"
```
byłyby nadmiarowe
*   Wersja uciąglona - relaksacja liniowa problemu klasycznego (`ints = False` oraz `classical = True`):
```python
  ile_rzeczy = plp.LpVariable(
      "x",
      [item.Nazwa for item in rzeczy],
      lowBound = 0,
      upBound = 1,
      cat = plp.LpContinuous
  )
```
Tutaj nie potrzeba ograniczenia:
```python
    for item in rzeczy:
      model_knapsack += ile_rzeczy[item.Nazwa] <= 1, f"ile_max_{item.Nazwa}"
```
gdyż ograniczenie górne na zmienne zostalo zadane z użyciem `upBound = 1` w definicji `x`
*   Wersja uciąglona z możliwością użycia większej liczby sztuk niż jeden (`ints = False` oraz `classical = False`):
```python
  ile_rzeczy = plp.LpVariable(
      "x",
      [item.Nazwa for item in rzeczy],
      lowBound = 0,
      cat = plp.LpContinuous
  )

  ...

    for item in rzeczy:
      model_knapsack += ile_rzeczy[item.Nazwa] <= item.Liczba_max, f"ile_max_{item.Nazwa}"
```

Po przeanalizowaniu kodu proszę wykonać poniższy kod i zastanowić się, dlaczego uzyskujemy wyniki o określonych wartościach - przy użyciu każdej z możliwych kombinacji wartości flag logicznych `ints` oraz `classical`.

1. Proszę rozwiązać różne wersje problemu z podanymi wcześniej przykładowymi danymi. Poniżej umieszczone cztery możliwe wersje w określonej kolejności (uszeregowane ze względu na wartość optymalną funkcji celu).
2. Proszę porównać rozwiązanie optymalne zadania z użyciem zmiennych dyskretnych oraz w sytuacji zrelaksowania tych zmiennych - jest to tzw. **relaksacja liniowa**, która polega na uciągleniu zmiennych (tj. zamiany typu na ciągły).
3. Proszę również sprawdzić, jak wygląda rozwiązanie w przypadku zrelaksowania ograniczenia górnego na wartość zmiennych decyzyjnych (zmieniamy ograniczenie górne wartości z 1 na wartość `Liczba_max`).
4. Warto również zwrócić uwagę na czas rozwiązania.

In [ ]:
#@markdown Problem klasyczny, tj. same zmienne binarne

# Moglibyśmy po prostu: knapsack(RZECZY, NOSNOSC_PLECAKA, log_output=True, float_precision=2)
model_knapsack = knapsack(RZECZY, NOSNOSC_PLECAKA, ints=True, classical=True, msg = True, mip = True)

Warto wybrać najszybszy solwer i liczyć z jego udziałem. Inna sprawa, że dla tak małego problemu to pomiar raczej jest mało wiarygodny :) 

In [ ]:
#@markdown W przypadku programowania dyskretnego możemy pamiętać o użyciu lepszych solwerów niż domyślny - problemy dyskretne (duże) mogą się naprawdę dłuuuuugo liczyć...

for solw in plp.listSolvers(onlyAvailable=True):
  print(f'Solwer: {solw}')
  solution = model_knapsack.solve(solver = plp.getSolver(solw))
  if solution == 1:
      print(f'Czas rozwiązywania: {model_knapsack.solutionTime}')
  else:
      print('Brak rozwiązania')
  print()

In [ ]:
#@markdown Problem zrelaksowany, ale zmienne $x_i \in [0,1]$

model_knapsack = knapsack(RZECZY, NOSNOSC_PLECAKA, ints=False, classical=True, msg = True, mip = True)

In [ ]:
#@markdown Zmienne całkowitoliczbowe: problem w wersji pełnej, której sformułowanie matematyczne podano wyżej

model_knapsack = knapsack(RZECZY, NOSNOSC_PLECAKA, ints=True, classical=False, msg = True, mip = True)

In [ ]:
#@markdown Problem zrelaksowany i dopuszczamy $x_i \geq 1$

model_knapsack = knapsack(RZECZY, NOSNOSC_PLECAKA, ints=False, classical=False, msg = True, mip = True)

### Zadanie 1: Eksperymentowanie z problemem plecakowym

Proszę:

1. Rozszerzyć sformułowanie problemu plecakowego w taki sposób, żeby uwzględnić nie tylko wagę - ale również rozmiary umieszczanych w plecaku przedmiotów. To znaczy:
  *   założyć, że każdy obiekt ma określoną szerokość;
  *   założyć, że plecak pomieści obiekty o określonej sumarycznej szerokości i jednocześnie o określonej sumarycznej masie.
2. Proszę rozwiązać tak rozszerzony model dla wymyślonych przez siebie danych (im bliższe rzeczywistości, tym intuicyjnie lepiej zrozumiałe rozwiązanie).

In [ ]:
#@title ROZWIĄZANIE ZADANIA 1

# TODO

## Przygotowanie do Q&A

Proszę zamieścić na [forum](https://upel.agh.edu.pl/mod/forum/view.php?id=1512) wszelkie pytania i wątpliwości, które mają być przedmiotem dyskusji w trakcie zajęć.